In [5]:
!pip install transformers==4.44.2 joblib==1.4.2 scikit-learn==1.6.0 numpy==1.26.4 pandas==2.2.3 scipy==1.13.1 seaborn==0.13.2 tqdm==4.66.5 

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.7/43.7 kB 1.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.6/60.6 kB 4.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.6/57.6 kB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.5/9.5 MB 83.3 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 301.8/301.8 kB 21.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.5/13.5 MB 103.6 MB/s eta 0:00:0000:010:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 38.6/38.6 MB 45.2 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.9/294.9 kB 20.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.4/78.4 kB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 81.8 MB/s eta 0:00:00:00:01
  Attempting uninstall: tqdm
    Found existing installation: tqdm 4.67.1
    Uninstalling tqdm-4.67.1:
      Successfully uninstalled tqdm-4.67.1
  Attempting uninstall:

In [32]:
import os
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
import numpy as np
from transformers import AutoTokenizer, AutoModelForSequenceClassification, AdamW
import torch
from torch.utils.data import DataLoader, Dataset


# Load datasets
train_df = pd.read_csv('/kaggle/input/rrck-set/Train_RRCK.csv')
train_df = train_df[['ID', 'SMILES', 'Permeability']]
test_df = pd.read_csv('/kaggle/input/rrck-set/Test_RRCK.csv')
test_df = test_df[['ID', 'SMILES', 'Permeability']]

In [33]:
tokenizer = AutoTokenizer.from_pretrained("seyonec/PubChem10M_SMILES_BPE_450k")
model = AutoModelForSequenceClassification.from_pretrained('seyonec/PubChem10M_SMILES_BPE_450k', num_labels=1)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)

/usr/local/lib/python3.11/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at seyonec/PubChem10M_SMILES_BPE_450k and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [34]:
# Custom dataset class
class SMILESDataset(Dataset):
    def __init__(self, dataframe, tokenizer, max_length=325):
        self.tokenizer = tokenizer
        self.dataframe = dataframe
        self.max_length = max_length

    def __len__(self):
        return len(self.dataframe)

    def __getitem__(self, idx):
        smiles = self.dataframe.iloc[idx]['SMILES']
        permeability = self.dataframe.iloc[idx]['Permeability']
        inputs = self.tokenizer(smiles, return_tensors='pt', padding="max_length", truncation=True, max_length=self.max_length)
        
        input_ids = inputs['input_ids'].squeeze(0)  # Shape: (sequence_length,)
        attention_mask = inputs['attention_mask'].squeeze(0)  # Shape: (sequence_length,)
        
        return {
            'input_ids': input_ids,
            'attention_mask': attention_mask,
            'labels': torch.tensor(permeability, dtype=torch.float)
        }

In [35]:
# datasets
train_dataset = SMILESDataset(train_df, tokenizer)
test_dataset = SMILESDataset(test_df, tokenizer)
batch_size = 16
# data loaders
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

In [36]:
optimizer = torch.optim.AdamW(model.parameters(), lr=5e-5)
num_epochs = 20

In [37]:
# Training loop
from tqdm import tqdm
for epoch in range(num_epochs):
    print(f"Entered Epoch {epoch + 1}")
    model.train()
    train_loss = 0

    for batch in tqdm(train_loader, desc=f'Training Epoch {epoch + 1}/{num_epochs}', unit='batch'):
        optimizer.zero_grad()

        # Move all batch tensors to device
        batch = {k: v.to(device) for k, v in batch.items()}
        labels = batch["labels"].unsqueeze(1)  # still shape: (batch_size, 1)

        # Forward pass
        outputs = model(
            input_ids=batch["input_ids"],
            attention_mask=batch["attention_mask"],
            labels=labels
        )
        loss = outputs.loss
        train_loss += loss.item()

        # Backprop and optimizer step
        loss.backward()
        optimizer.step()

    avg_train_loss = train_loss / len(train_loader)
    print(f'Epoch {epoch + 1}/{num_epochs} - Train Loss: {avg_train_loss:.4f}')

Entered Epoch 1


Training Epoch 1/20: 100%|██████████| 9/9 [00:04<00:00,  2.11batch/s]


Epoch 1/20 - Train Loss: 8.9178
Entered Epoch 2


Training Epoch 2/20: 100%|██████████| 9/9 [00:04<00:00,  1.98batch/s]


Epoch 2/20 - Train Loss: 0.5941
Entered Epoch 3


...

Epoch 18/20 - Train Loss: 0.1343
Entered Epoch 19


Training Epoch 19/20: 100%|██████████| 9/9 [00:04<00:00,  2.13batch/s]


Epoch 19/20 - Train Loss: 0.1816
Entered Epoch 20


Training Epoch 20/20: 100%|██████████| 9/9 [00:04<00:00,  2.11batch/s]

Epoch 20/20 - Train Loss: 0.1202


In [38]:
# Saving the model after training
model_name = 'PubChem10M_SMILES_BPE_450k_model_1_rrck'
model_save_path = f'/kaggle/working/{model_name}'
os.makedirs(model_save_path, exist_ok=True)

tokenizer.save_pretrained(model_save_path)
model.save_pretrained(model_save_path)

print(f'Model and tokenizer saved to {model_save_path}')

Model and tokenizer saved to /kaggle/working/PubChem10M_SMILES_BPE_450k_model_1_rrck


In [39]:
from scipy.stats import pearsonr, spearmanr

model.eval()
test_loss = 0
test_true_labels = []
predictions = []

with torch.no_grad():
    for batch in tqdm(test_loader, desc='Testing', unit='batch'):
      
        input_ids = batch["input_ids"].to(device)
        attention_mask = batch["attention_mask"].to(device)
        labels = batch["labels"].unsqueeze(1).to(device).float()

        outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss
        test_loss += loss.item()

        test_true_labels.extend(labels.cpu().numpy())
        preds = outputs.logits.squeeze().cpu().numpy()  
        predictions.extend(preds)

# Final test loss
avg_test_loss = test_loss / len(test_loader)
print(f'Test Loss: {avg_test_loss:.4f}')

test_true_labels = np.array(test_true_labels).flatten()
predictions = np.array(predictions)
print(test_true_labels.shape)
print(predictions.shape)

# Performance metrics
mse = mean_squared_error(test_true_labels, predictions)
rmse = np.sqrt(mse)
mae = mean_absolute_error(test_true_labels, predictions)
r2 = r2_score(test_true_labels, predictions)
PCC,_ = pearsonr(test_true_labels, predictions)
SCC,_ = spearmanr(test_true_labels, predictions)

# Print performance metrics
print(f'Mean Squared Error: {mse:.4f}')
print(f'Root Mean Squared Error: {rmse:.4f}')
print(f'Mean Absolute Error: {mae:.4f}')
print(f'R^2 Score: {r2:.4f}')
print(f'Pearson Correlation Coefficient: {PCC:.4f}')
print(f'Spearman Correlation Coefficient: {SCC:.4f}')

# Print hyperparameters
print("Hyperparameters:")
print(f"Learning Rate: {5e-5}")
print(f"Batch Size: 16")
print(f"Epochs: {num_epochs}")

Testing: 100%|██████████| 3/3 [00:00<00:00,  7.81batch/s]

Test Loss: 0.5002
(36,)
(36,)
Mean Squared Error: 0.6452
Root Mean Squared Error: 0.8033
Mean Absolute Error: 0.6009
R^2 Score: 0.1074
Pearson Correlation Coefficient: 0.4246
Spearman Correlation Coefficient: 0.5184
Hyperparameters:
Learning Rate: 5e-05
Batch Size: 16
Epochs: 20


In [40]:
import os
import pandas as pd
import numpy as np
import torch
from torch.utils.data import Dataset, DataLoader
from torch.optim import AdamW
from transformers import AutoTokenizer, AutoModel

from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

model_name = 'PubChem10M_SMILES_BPE_450k_model_1_rrck'
model_save_path = f'/kaggle/working/{model_name}'

if not os.path.exists(model_save_path):
    raise FileNotFoundError(f"The model directory {model_save_path} does not exist.")

# Load tokenizer and model
tokenizer = AutoTokenizer.from_pretrained(model_save_path, trust_remote_code=True)
model = AutoModel.from_pretrained(model_save_path, trust_remote_code=True).to(device)

Some weights of RobertaModel were not initialized from the model checkpoint at /kaggle/working/PubChem10M_SMILES_BPE_450k_model_1_rrck and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [41]:
# Load your datasets
train_df = pd.read_csv('/kaggle/input/rrck-set/Train_RRCK.csv')
train_df = train_df[['ID', 'SMILES', 'Permeability']]
test_df = pd.read_csv('/kaggle/input/rrck-set/Test_RRCK.csv')
test_df = test_df[['ID', 'SMILES', 'Permeability']]

In [42]:
train_encodings = tokenizer(list(train_df['SMILES']), truncation=True, padding=True, max_length=325, return_tensors="pt")
test_encodings = tokenizer(list(test_df['SMILES']), truncation=True, padding=True, max_length=325, return_tensors="pt")

In [43]:
from tqdm import tqdm 
batch_size = 16 

def generate_embeddings(encodings, batch_size):
    embeddings = []
    model.eval() 
    with torch.no_grad():
        for i in tqdm(range(0, len(encodings['input_ids']), batch_size), desc="Processing batches"):
            batch = {key: val[i:i + batch_size].to(device) for key, val in encodings.items()}  
            outputs = model(**batch)
            embeddings.append(outputs.last_hidden_state)
    return torch.cat(embeddings, dim=0)


In [44]:
train_embeddings = generate_embeddings(train_encodings, batch_size)
print(train_embeddings.shape)
train_embeddings = torch.mean(train_embeddings, dim=1)
print(train_embeddings.shape)

Processing batches: 100%|██████████| 9/9 [00:00<00:00, 11.75it/s]

torch.Size([140, 198, 768])
torch.Size([140, 768])


In [45]:
column_names = [f'x_fine_emb_pubchem{i}' for i in range(train_embeddings.shape[1])]
embeddings_df = pd.DataFrame(data=train_embeddings.cpu().numpy(), columns=column_names)
train_data = pd.concat([train_df, embeddings_df], axis=1)

In [46]:
test_embeddings = generate_embeddings(test_encodings, batch_size)
print(test_embeddings.shape)
test_embeddings = torch.mean(test_embeddings, dim=1)
print(test_embeddings.shape)

Processing batches: 100%|██████████| 3/3 [00:00<00:00, 15.13it/s]


torch.Size([36, 201, 768])
torch.Size([36, 768])


In [47]:
column_names = [f'x_fine_emb_pubchem{i}' for i in range(test_embeddings.shape[1])]
embeddings_df = pd.DataFrame(data=test_embeddings.cpu().numpy(), columns=column_names)
test_data = pd.concat([test_df, embeddings_df], axis=1)

In [48]:
train_data.to_csv("/kaggle/working/Train_PubChem10M_SMILES_BPE_450k_model_1_fine_tuned_embeddings_rrck.csv",index=False)
test_data.to_csv("/kaggle/working/Test_PubChem10M_SMILES_BPE_450k_model_1_fine_tuned_embeddings_rrck.csv",index=False)

In [49]:
import numpy as np
import pandas as pd
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor, AdaBoostRegressor, ExtraTreesRegressor
from sklearn.linear_model import LinearRegression, LogisticRegression  # LogisticRegression is not used for regression
from sklearn.neighbors import KNeighborsRegressor
from sklearn.svm import SVR
from sklearn.neural_network import MLPRegressor
import xgboost as xgb
import lightgbm as lgb
from sklearn.model_selection import KFold, GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from scipy.stats import pearsonr, spearmanr
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt

In [50]:
train_data = pd.read_csv("/kaggle/working/Train_PubChem10M_SMILES_BPE_450k_model_1_fine_tuned_embeddings_rrck.csv")
test_data = pd.read_csv("/kaggle/working/Test_PubChem10M_SMILES_BPE_450k_model_1_fine_tuned_embeddings_rrck.csv")

In [51]:
def train_and_test_predict(models, X_train, y_train, X_test, y_test):
    kf = KFold(n_splits=5, shuffle=True, random_state=42)
    results = {}
    predictions = []  

    for model in models:
        model_name = model.__class__.__name__
        predictions_train = []
        actual_y_train = []

        test_predictions_folds = []

        

        for train_index, val_index in kf.split(X_train):
            X_train_fold, X_val_fold = X_train.iloc[train_index], X_train.iloc[val_index]
            y_train_fold, y_val_fold = y_train.iloc[train_index], y_train.iloc[val_index]

            model.fit(X_train_fold, y_train_fold)

            y_pred_fold = model.predict(X_val_fold)
            y_pred_fold = np.clip(y_pred_fold, -10, -3.9)
            predictions_train.extend(y_pred_fold)
            actual_y_train.extend(y_val_fold)

            predictions_test_fold = model.predict(X_test)
            predictions_test_fold = np.clip(predictions_test_fold, -10, -3.9)
            test_predictions_folds.append(predictions_test_fold)


        mse_train = mean_squared_error(actual_y_train, predictions_train)
        mae_train = mean_absolute_error(actual_y_train, predictions_train)
        rmse_train = np.sqrt(mse_train)
        r2_train = r2_score(actual_y_train, predictions_train)
        pearson_train, _ = pearsonr(actual_y_train, predictions_train)
        spearman_train, _ = spearmanr(actual_y_train, predictions_train)


        predictions_test_mean = np.mean(test_predictions_folds, axis=0)
        predictions_test_std = np.std(test_predictions_folds, axis=0)

        mse_test = mean_squared_error(y_test, predictions_test_mean)
        mae_test = mean_absolute_error(y_test, predictions_test_mean)
        rmse_test = np.sqrt(mse_test)
        r2_test = r2_score(y_test, predictions_test_mean)
        print(r2_test)
        pearson_test, _ = pearsonr(y_test, predictions_test_mean)
        spearman_test, _ = spearmanr(y_test, predictions_test_mean)
        
        

        predictions.append({
            'Model': model_name,
            'Y Train pred': predictions_train,
            'Y Test actual': y_test,
            'Test prediction folds': test_predictions_folds,
            'Test Predictions Mean': predictions_test_mean,
            'Test Predictions Std': predictions_test_std,

        })

        results[model_name] = {
            'Train MSE (5 fold cv)': f"{mse_train:.4f}",
            'Train MAE (5 fold cv)': f"{mae_train:.4f}",
            'Train RMSE (5 fold cv)': f"{rmse_train:.4f}",
            'Train RMSE (5 fold cv)': f"{rmse_train:.4f}",
            'Train R2 (5 fold cv)': f"{r2_train:.4f}",
            'Train PCC (5 fold cv)': f"{pearson_train:.4f}",
            'Train SCC (5 fold cv)': f"{spearman_train:.4f}",
            'Test MSE': f"{mse_test:.4f}",
            'Test MAE': f"{mae_test:.4f}",
            'Test RMSE': f"{rmse_test:.4f}",
            'Test R2': f"{r2_test:.4f}",
            'Test Pearson Correlation': f"{pearson_test:.4f}",
            'Test Spearman Correlation': f"{spearman_test:.4f}",
        }

    results_df = pd.DataFrame(results).T
    predictions_df = pd.DataFrame(predictions)

    return results_df, predictions_df



In [52]:
X_train = train_data.drop(['ID','SMILES','Permeability'],axis=1)
y_train = train_data['Permeability']
print("X_train shape: ",X_train.shape)
print("y_train shape: ",y_train.shape)
print("XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX")

X_test = test_data.drop(['ID','SMILES','Permeability'],axis=1)
y_test = test_data['Permeability']
print("X_test shape: ",X_test.shape)
print("y_test shape: ",y_test.shape)
print("XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX")
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)
X_train = pd.DataFrame(X_train_scaled, columns=X_train.columns, index=X_train.index)
X_test = pd.DataFrame(X_test_scaled, columns=X_test.columns, index=X_test.index)
models = [
    lgb.LGBMRegressor(objective='regression',metric='rmse',boosting_type='gbdt',num_leaves=31,learning_rate=0.05,random_state=42),
    DecisionTreeRegressor(random_state=42),
    RandomForestRegressor(n_jobs=-1, random_state=42),
    GradientBoostingRegressor(random_state=42),
    AdaBoostRegressor(random_state=42),
    xgb.XGBRegressor(random_state=42),
    ExtraTreesRegressor(n_jobs=-1, n_estimators=100, random_state=42),
    LinearRegression(), 
    KNeighborsRegressor(n_neighbors=3),
    SVR(),  
    MLPRegressor(random_state=42)
]
result_df, prediction_df = train_and_test_predict(models, X_train,y_train, X_test,  y_test)
result_df

X_train shape:  (140, 768)
y_train shape:  (140,)
XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX
X_test shape:  (36, 768)
y_test shape:  (36,)
XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.001309 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 29952
[LightGBM] [Info] Number of data points in the train set: 112, number of used features: 768
[LightGBM] [Info] Start training from score -5.550491
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further sp

,Train MSE (5 fold cv),Train MAE (5 fold cv),Train RMSE (5 fold cv),Train R2 (5 fold cv),Train PCC (5 fold cv),Train SCC (5 fold cv),Test MSE,Test MAE,Test RMSE,Test R2,Test Pearson Correlation,Test Spearman Correlation
LGBMRegressor,0.1316,0.2669,0.3628,0.7064,0.8408,0.8493,0.6306,0.5465,0.7941,0.1276,0.5136,0.5885
DecisionTreeRegressor,0.2187,0.3551,0.4676,0.5122,0.7569,0.7678,0.5932,0.5315,0.7702,0.1793,0.5290,0.6164
RandomForestRegressor,0.1364,0.2820,0.3693,0.6958,0.8373,0.8543,0.6505,0.5444,0.8065,0.1001,0.4780,0.5655
GradientBoostingRegressor,0.1358,0.2814,0.3685,0.6970,0.8361,0.8500,0.6215,0.5360,0.7883,0.1402,0.5097,0.5975
AdaBoostRegressor,0.1433,0.2939,0.3786,0.6803,0.8288,0.8523,0.6520,0.5453,0.8075,0.0979,0.4689,0.5506
XGBRegressor,0.1788,0.3163,0.4229,0.6011,0.7840,0.8018,0.6265,0.5330,0.7915,0.1333,0.5118,0.6067
ExtraTreesRegressor,0.1229,0.2637,0.3505,0.7259,0.8528,0.8628,0.6638,0.5397,0.8147,0.0817,0.4806,0.5597
LinearRegression,0.5177,0.5319,0.7195,-0.1548,0.6279,0.6672,0.6535,0.5934,0.8084,0.0960,0.4943,0.5822
KNeighborsRegressor,0.1799,0.3116,0.4242,0.5986,0.7801,0.7906,0.6012,0.5197,0.7754,0.1682,0.5655,0.6530
SVR,0.1235,0.2580,0.3514,0.7246,0.8546,0.8699,0.7064,0.5769,0.8405,0.0227,0.4462,0.5406


In [53]:
result_df

,Train MSE (5 fold cv),Train MAE (5 fold cv),Train RMSE (5 fold cv),Train R2 (5 fold cv),Train PCC (5 fold cv),Train SCC (5 fold cv),Test MSE,Test MAE,Test RMSE,Test R2,Test Pearson Correlation,Test Spearman Correlation
LGBMRegressor,0.1316,0.2669,0.3628,0.7064,0.8408,0.8493,0.6306,0.5465,0.7941,0.1276,0.5136,0.5885
DecisionTreeRegressor,0.2187,0.3551,0.4676,0.5122,0.7569,0.7678,0.5932,0.5315,0.7702,0.1793,0.5290,0.6164
RandomForestRegressor,0.1364,0.2820,0.3693,0.6958,0.8373,0.8543,0.6505,0.5444,0.8065,0.1001,0.4780,0.5655
GradientBoostingRegressor,0.1358,0.2814,0.3685,0.6970,0.8361,0.8500,0.6215,0.5360,0.7883,0.1402,0.5097,0.5975
AdaBoostRegressor,0.1433,0.2939,0.3786,0.6803,0.8288,0.8523,0.6520,0.5453,0.8075,0.0979,0.4689,0.5506
XGBRegressor,0.1788,0.3163,0.4229,0.6011,0.7840,0.8018,0.6265,0.5330,0.7915,0.1333,0.5118,0.6067
ExtraTreesRegressor,0.1229,0.2637,0.3505,0.7259,0.8528,0.8628,0.6638,0.5397,0.8147,0.0817,0.4806,0.5597
LinearRegression,0.5177,0.5319,0.7195,-0.1548,0.6279,0.6672,0.6535,0.5934,0.8084,0.0960,0.4943,0.5822
KNeighborsRegressor,0.1799,0.3116,0.4242,0.5986,0.7801,0.7906,0.6012,0.5197,0.7754,0.1682,0.5655,0.6530
SVR,0.1235,0.2580,0.3514,0.7246,0.8546,0.8699,0.7064,0.5769,0.8405,0.0227,0.4462,0.5406


In [54]:
prediction_df

,Model,Y Train pred,Y Test actual,Test prediction folds,Test Predictions Mean,Test Predictions Std
0,LGBMRegressor,"[-6.1054651180084, -6.361619216003109, -6.0844...",0 -6.340 1 -5.760 2 -8.000 3 -6.46...,"[[-6.1423422534819245, -6.06453568642041, -5.8...","[-6.212355527885379, -6.015832798061995, -5.97...","[0.04610132017789668, 0.04596563632613679, 0.1..."
1,DecisionTreeRegressor,"[-5.95, -6.5, -6.46, -5.21, -5.92, -5.57, -6.1...",0 -6.340 1 -5.760 2 -8.000 3 -6.46...,"[[-5.96, -5.96, -6.0, -6.46, -5.48, -5.21, -6....","[-6.438000000000001, -5.92, -6.072, -6.2540000...","[0.580840769918917, 0.04098780306383839, 0.470..."
2,RandomForestRegressor,"[-6.065199999999999, -6.288299999999998, -6.22...",0 -6.340 1 -5.760 2 -8.000 3 -6.46...,"[[-6.347699999999999, -6.0486999999999975, -5....","[-6.267410000000001, -5.984890000000001, -5.84...","[0.08240964992038223, 0.03436513349311906, 0.1..."
3,GradientBoostingRegressor,"[-5.9749838081469875, -6.311507314101484, -6.1...",0 -6.340 1 -5.760 2 -8.000 3 -6.46...,"[[-6.319573591664155, -5.968667256215465, -5.7...","[-6.329655026770734, -5.93233751723744, -5.979...","[0.11799735404656887, 0.025384913469978017, 0...."
4,AdaBoostRegressor,"[-6.128500000000001, -6.066499999999999, -6.02...",0 -6.340 1 -5.760 2 -8.000 3 -6.46...,"[[-6.411818181818179, -6.122931034482757, -5.5...","[-6.26983736378564, -6.0549790339667995, -5.87...","[0.1531929311542744, 0.04827235114651696, 0.18..."
5,XGBRegressor,"[-5.95468, -6.3890743, -5.8004394, -5.2590213,...",0 -6.340 1 -5.760 2 -8.000 3 -6.46...,"[[-6.077399, -5.9501176, -5.746871, -6.3135204...","[-6.1044455, -5.933655, -5.908435, -6.3965006,...","[0.10180307, 0.025337119, 0.18753104, 0.084194..."
6,ExtraTreesRegressor,"[-6.002149999999999, -6.319900000000002, -6.00...",0 -6.340 1 -5.760 2 -8.000 3 -6.46...,"[[-6.26865, -5.966700000000002, -5.76375000000...","[-6.28964, -5.92436, -5.820940000000004, -6.33...","[0.040895566996926, 0.02233540686891574, 0.068..."
7,LinearRegression,"[-5.894731227541739, -5.689643033830869, -5.48...",0 -6.340 1 -5.760 2 -8.000 3 -6.46...,"[[-6.660566072809806, -5.892388034166153, -5.7...","[-6.267182031301308, -5.885266180380028, -5.99...","[0.3866324280900806, 0.053503544725336416, 0.2..."
8,KNeighborsRegressor,"[-6.246666666666667, -6.073333333333333, -5.76...",0 -6.340 1 -5.760 2 -8.000 3 -6.46...,"[[-6.246666666666667, -6.246666666666667, -5.7...","[-6.126666666666667, -6.126666666666667, -5.88...","[0.1219653867109665, 0.1219653867109665, 0.229..."
9,SVR,"[-6.029905130184228, -6.121730929705714, -5.86...",0 -6.340 1 -5.760 2 -8.000 3 -6.46...,"[[-6.15726818307571, -6.0327743508601905, -5.7...","[-6.065831459859936, -5.995061382622513, -5.77...","[0.06513092677370014, 0.019421895202050753, 0...."


In [55]:
result_df.to_csv('/kaggle/working/Results_PubChem10M_SMILES_BPE_450k_model_1_fine_tuned_embeddings_rrck.csv')
prediction_df.to_csv('/kaggle/working/Prediction_data_PubChem10M_SMILES_BPE_450k_model_1_fine_tuned_embeddings_rrck.csv')

In [56]:
result_df

,Train MSE (5 fold cv),Train MAE (5 fold cv),Train RMSE (5 fold cv),Train R2 (5 fold cv),Train PCC (5 fold cv),Train SCC (5 fold cv),Test MSE,Test MAE,Test RMSE,Test R2,Test Pearson Correlation,Test Spearman Correlation
LGBMRegressor,0.1316,0.2669,0.3628,0.7064,0.8408,0.8493,0.6306,0.5465,0.7941,0.1276,0.5136,0.5885
DecisionTreeRegressor,0.2187,0.3551,0.4676,0.5122,0.7569,0.7678,0.5932,0.5315,0.7702,0.1793,0.5290,0.6164
RandomForestRegressor,0.1364,0.2820,0.3693,0.6958,0.8373,0.8543,0.6505,0.5444,0.8065,0.1001,0.4780,0.5655
GradientBoostingRegressor,0.1358,0.2814,0.3685,0.6970,0.8361,0.8500,0.6215,0.5360,0.7883,0.1402,0.5097,0.5975
AdaBoostRegressor,0.1433,0.2939,0.3786,0.6803,0.8288,0.8523,0.6520,0.5453,0.8075,0.0979,0.4689,0.5506
XGBRegressor,0.1788,0.3163,0.4229,0.6011,0.7840,0.8018,0.6265,0.5330,0.7915,0.1333,0.5118,0.6067
ExtraTreesRegressor,0.1229,0.2637,0.3505,0.7259,0.8528,0.8628,0.6638,0.5397,0.8147,0.0817,0.4806,0.5597
LinearRegression,0.5177,0.5319,0.7195,-0.1548,0.6279,0.6672,0.6535,0.5934,0.8084,0.0960,0.4943,0.5822
KNeighborsRegressor,0.1799,0.3116,0.4242,0.5986,0.7801,0.7906,0.6012,0.5197,0.7754,0.1682,0.5655,0.6530
SVR,0.1235,0.2580,0.3514,0.7246,0.8546,0.8699,0.7064,0.5769,0.8405,0.0227,0.4462,0.5406


In [57]:
prediction_df

,Model,Y Train pred,Y Test actual,Test prediction folds,Test Predictions Mean,Test Predictions Std
0,LGBMRegressor,"[-6.1054651180084, -6.361619216003109, -6.0844...",0 -6.340 1 -5.760 2 -8.000 3 -6.46...,"[[-6.1423422534819245, -6.06453568642041, -5.8...","[-6.212355527885379, -6.015832798061995, -5.97...","[0.04610132017789668, 0.04596563632613679, 0.1..."
1,DecisionTreeRegressor,"[-5.95, -6.5, -6.46, -5.21, -5.92, -5.57, -6.1...",0 -6.340 1 -5.760 2 -8.000 3 -6.46...,"[[-5.96, -5.96, -6.0, -6.46, -5.48, -5.21, -6....","[-6.438000000000001, -5.92, -6.072, -6.2540000...","[0.580840769918917, 0.04098780306383839, 0.470..."
2,RandomForestRegressor,"[-6.065199999999999, -6.288299999999998, -6.22...",0 -6.340 1 -5.760 2 -8.000 3 -6.46...,"[[-6.347699999999999, -6.0486999999999975, -5....","[-6.267410000000001, -5.984890000000001, -5.84...","[0.08240964992038223, 0.03436513349311906, 0.1..."
3,GradientBoostingRegressor,"[-5.9749838081469875, -6.311507314101484, -6.1...",0 -6.340 1 -5.760 2 -8.000 3 -6.46...,"[[-6.319573591664155, -5.968667256215465, -5.7...","[-6.329655026770734, -5.93233751723744, -5.979...","[0.11799735404656887, 0.025384913469978017, 0...."
4,AdaBoostRegressor,"[-6.128500000000001, -6.066499999999999, -6.02...",0 -6.340 1 -5.760 2 -8.000 3 -6.46...,"[[-6.411818181818179, -6.122931034482757, -5.5...","[-6.26983736378564, -6.0549790339667995, -5.87...","[0.1531929311542744, 0.04827235114651696, 0.18..."
5,XGBRegressor,"[-5.95468, -6.3890743, -5.8004394, -5.2590213,...",0 -6.340 1 -5.760 2 -8.000 3 -6.46...,"[[-6.077399, -5.9501176, -5.746871, -6.3135204...","[-6.1044455, -5.933655, -5.908435, -6.3965006,...","[0.10180307, 0.025337119, 0.18753104, 0.084194..."
6,ExtraTreesRegressor,"[-6.002149999999999, -6.319900000000002, -6.00...",0 -6.340 1 -5.760 2 -8.000 3 -6.46...,"[[-6.26865, -5.966700000000002, -5.76375000000...","[-6.28964, -5.92436, -5.820940000000004, -6.33...","[0.040895566996926, 0.02233540686891574, 0.068..."
7,LinearRegression,"[-5.894731227541739, -5.689643033830869, -5.48...",0 -6.340 1 -5.760 2 -8.000 3 -6.46...,"[[-6.660566072809806, -5.892388034166153, -5.7...","[-6.267182031301308, -5.885266180380028, -5.99...","[0.3866324280900806, 0.053503544725336416, 0.2..."
8,KNeighborsRegressor,"[-6.246666666666667, -6.073333333333333, -5.76...",0 -6.340 1 -5.760 2 -8.000 3 -6.46...,"[[-6.246666666666667, -6.246666666666667, -5.7...","[-6.126666666666667, -6.126666666666667, -5.88...","[0.1219653867109665, 0.1219653867109665, 0.229..."
9,SVR,"[-6.029905130184228, -6.121730929705714, -5.86...",0 -6.340 1 -5.760 2 -8.000 3 -6.46...,"[[-6.15726818307571, -6.0327743508601905, -5.7...","[-6.065831459859936, -5.995061382622513, -5.77...","[0.06513092677370014, 0.019421895202050753, 0...."
